### Coordenadas Barrios

En este libro, consigo crear un fichero con los barrios y distritos de Madrid usando los ficheros del [ayuntamiento de madrid con todos los Distritos de Madrid](https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=46b55cde99be2410VgnVCM1000000b205a0aRCRD&).

Para ello, uso Geopandas y la API Geocoding de Google.

In [66]:
#importo geopandas para poder leer el fichero .shp
import geopandas as gpd

#con pandas manipulamos el dataframe
import pandas as pd

#para hacer solicitudes a la API
import requests

In [67]:
#con geopandas convierto el fichero shp en un dataframe
path_barrios = "C:/Users/Sergio/Python_projects/Madrid/Barrios/BARRIOS.shp"
barrios = gpd.read_file(path_barrios)
barrios.head()

,OBJECTID,geodb_oid,CODDIS,NOMDIS,CODBAR,CODDISTRIT,CODBARRIO,NOMBRE,ORIG_FID,geometry
0,108,108,17,Villaverde,172,17,17-2,San Cristobal,107,"POLYGON ((441930.8668000005 4466853.1887, 4419..."
1,109,109,17,Villaverde,173,17,17-3,Butarque,108,"POLYGON ((444144.8566044134 4464473.210504748,..."
2,111,111,17,Villaverde,175,17,17-5,Los Angeles,110,"POLYGON ((441147.7280000008 4466374.483400001,..."
3,110,110,17,Villaverde,174,17,17-4,Los Rosales,109,"POLYGON ((442098.8264462068 4468081.978228984,..."
4,107,107,17,Villaverde,171,17,17-1,"Villaverde Alto, Casco Histórico de Villaverde",106,"POLYGON ((440129.2758234279 4468362.077814397,..."


In [68]:
#necesito el barrio y los distritos
madrid_data = barrios[['NOMDIS', 'NOMBRE']]
madrid_data = madrid_data.rename(columns={'NOMDIS':'borough',
                                          'NOMBRE':'neighborhood'})

In [69]:
#asi es como queda por ahora
madrid_data.head()

,borough,neighborhood
0,Villaverde,San Cristobal
1,Villaverde,Butarque
2,Villaverde,Los Angeles
3,Villaverde,Los Rosales
4,Villaverde,"Villaverde Alto, Casco Histórico de Villaverde"


In [70]:
#creo una nueva columna que me va a facilitar la busqueda de las coordenadas
address = []
for bor, neigh in zip(madrid_data.borough, madrid_data.neighborhood):
    add = ('{}, {}, Madrid, Spain').format(neigh, bor)
    address.append(add)

In [71]:
#asi es como queda la direccion a usar en la API
dataset = pd.DataFrame({'address':address})
dataset.head()

,address
0,"San Cristobal, Villaverde, Madrid, Spain"
1,"Butarque, Villaverde, Madrid, Spain"
2,"Los Angeles, Villaverde, Madrid, Spain"
3,"Los Rosales, Villaverde, Madrid, Spain"
4,"Villaverde Alto, Casco Histórico de Villaverde..."


In [1]:
#for security purposes credentials are not visualized here
import config
google_api_key = config.google_api_key # google API key

In [72]:
#ahora vamos a usar la API de google para encontar las coordenadas
lat = []
lon = []
for address in address:
    api_key = google_api_key
    api_response = requests.get('https://maps.googleapis.com/maps/api/geocode/json?address={0}&key={1}'.format(address, api_key))
    api_response_dict = api_response.json()
    if api_response_dict['status'] == 'OK':
        latitude = api_response_dict['results'][0]['geometry']['location']['lat']
        longitude = api_response_dict['results'][0]['geometry']['location']['lng']
        lat.append(latitude)
        lon.append(longitude)

In [73]:
#convierto las listas en dataframes
latitude = pd.DataFrame({'latitude':lat})
longitude = pd.DataFrame({'longitude':lon})

In [74]:
#examino la latitud
latitude.head()

,latitude
0,40.340765
1,40.340605
2,40.354412
3,40.355939
4,40.351025


In [75]:
#añado las coordenadas al dataframe original
madrid_data = madrid_data.join(latitude)
madrid_data = madrid_data.join(longitude)
madrid_data.head()

,borough,neighborhood,latitude,longitude
0,Villaverde,San Cristobal,40.340765,-3.688948
1,Villaverde,Butarque,40.340605,-3.674808
2,Villaverde,Los Angeles,40.354412,-3.700363
3,Villaverde,Los Rosales,40.355939,-3.691342
4,Villaverde,"Villaverde Alto, Casco Histórico de Villaverde",40.351025,-3.692555


In [77]:
madrid_data.tail()

,borough,neighborhood,latitude,longitude
126,Fuencarral - El Pardo,Valverde,40.496110,-3.695941
127,Fuencarral - El Pardo,El Goloso,40.553385,-3.709355
128,Fuencarral - El Pardo,El Pardo,40.519069,-3.775877
129,Vicálvaro,Casco histórico de Vicálvaro,40.402694,-3.615925
130,Vicálvaro,El Cañaveral,40.401282,-3.552646


In [78]:
centro_data = madrid_data[madrid_data['borough'] == 'Centro']
centro_data.head

<bound method NDFrame.head of    borough neighborhood   latitude  longitude
49  Centro  Embajadores  40.408129  -3.707739
54  Centro       Cortes  40.414221  -3.699941
55  Centro          Sol  40.416663  -3.704169
59  Centro      Palacio  40.417955  -3.714312
63  Centro     Justicia  40.424531  -3.699293
65  Centro  Universidad  40.423372  -3.707792>

In [79]:
#guardamos el fichero
madrid_data.to_csv('coordenadas.csv')